# Audio FFT: Hardware vs Software Comparison

This notebook demonstrates the processing of a `.wav` audio file using the Vitis DSP Library FFT IP on the PYNQ-Z2 board. It compares the results and performance against a software implementation using NumPy.

In [ ]:
from pynq import Overlay, allocate
import numpy as np
import scipy.io.wavfile as wavfile
import matplotlib.pyplot as plt
import time
import os

# Load Overlay
overlay = Overlay("../build/overlay/fft.bit")
dma = overlay.axi_dma_0
fft_ip = overlay.fft_top_0

# Start the FFT IP (Auto-restart mode)
fft_ip.write(0x00, 0x81)

In [ ]:
# Configuration
N = 1024
SSR = 2
SCALE = 2**15

# Vectorized Packing Function (Optimized for Performance)
def pack_input_vectorized(signal):
    # signal: complex array of size N
    # Scale and cast to fixed point
    sig_fixed = (signal * SCALE).astype(np.complex64)
    real = np.real(sig_fixed).astype(np.int16).astype(np.uint32) & 0xFFFF
    imag = np.imag(sig_fixed).astype(np.int16).astype(np.uint32) & 0xFFFF
    
    # Combine 16-bit I/Q into 32-bit words
    sample_val = (imag << 16) | real
    
    # Split into SSR streams (Even/Odd)
    s0 = sample_val[0::2]
    s1 = sample_val[1::2]
    
    # Combine two 32-bit samples into one 64-bit word
    val = (s1.astype(np.uint64) << 32) | s0.astype(np.uint64)
    
    # Create buffer
    buf = allocate(shape=(N//SSR,), dtype=np.uint64)
    buf[:] = val
    return buf

# Vectorized Unpacking Function
def unpack_output_vectorized(buf):
    # buf: uint64 array of size N/SSR
    val = np.array(buf)
    
    # Extract 32-bit words
    s0_val = val & 0xFFFFFFFF
    s1_val = (val >> 32) & 0xFFFFFFFF
    
    # Extract 16-bit I/Q
    def extract_complex(v):
        real = np.int16(v & 0xFFFF)
        imag = np.int16((v >> 16) & 0xFFFF)
        return (real.astype(np.float32)/SCALE) + 1j * (imag.astype(np.float32)/SCALE)
    
    c0 = extract_complex(s0_val)
    c1 = extract_complex(s1_val)
    
    # Interleave to reconstruct N-point array
    output = np.zeros(N, dtype=np.complex64)
    output[0::2] = c0
    output[1::2] = c1
    return output

In [ ]:
# Load Audio File
AUDIO_PATH = "test_audio.wav"

if not os.path.exists(AUDIO_PATH):
    print(f"File {AUDIO_PATH} not found. Generating dummy audio signal.")
    fs = 44100
    duration = 2.0 # seconds
    t = np.linspace(0, duration, int(duration*fs))
    # Generate a mix of frequencies
    audio = 0.5 * np.sin(2*np.pi*440*t) + 0.3 * np.sin(2*np.pi*1000*t)
else:
    print(f"Loading {AUDIO_PATH}...")
    fs, audio = wavfile.read(AUDIO_PATH)
    # Convert to mono if stereo
    if len(audio.shape) > 1:
        audio = audio.mean(axis=1)
    # Normalize to -1.0 to 1.0
    if audio.dtype != np.float32 and audio.dtype != np.float64:
        audio = audio.astype(np.float32) / np.iinfo(audio.dtype).max

# Reshape audio into frames of size N
num_frames = len(audio) // N
audio_truncated = audio[:num_frames*N]
frames = audio_truncated.reshape((num_frames, N))
print(f"Processing {num_frames} frames of {N} samples each.")

In [ ]:
# Hardware FFT Processing Loop
def run_hw_fft(frames):
    num_frames = len(frames)
    in_buffers = []
    out_buffers = []
    
    print("Pre-packing data to isolate hardware performance...")
    # Pre-allocate and pack all frames to exclude Python overhead from benchmark
    for frame in frames:
        buf = allocate(shape=(N//SSR,), dtype=np.uint64)
        # Inline packing logic
        sig_fixed = (frame * SCALE).astype(np.complex64)
        real = np.real(sig_fixed).astype(np.int16).astype(np.uint32) & 0xFFFF
        imag = np.imag(sig_fixed).astype(np.int16).astype(np.uint32) & 0xFFFF
        sample_val = (imag << 16) | real
        val = (sample_val[1::2].astype(np.uint64) << 32) | sample_val[0::2].astype(np.uint64)
        buf[:] = val
        in_buffers.append(buf)
        
        # Output buffer
        out_buf = allocate(shape=(N//SSR,), dtype=np.uint64)
        out_buffers.append(out_buf)
    
    print("Starting Hardware Benchmark (DMA + Compute)...")
    start_time = time.perf_counter() # Use perf_counter for high precision
    
    for i in range(num_frames):
        dma.sendchannel.transfer(in_buffers[i])
        dma.recvchannel.transfer(out_buffers[i])
        dma.sendchannel.wait()
        dma.recvchannel.wait()
        
    end_time = time.perf_counter()
    hw_time = end_time - start_time
    
    # Unpack results for verification (outside timing loop)
    results = []
    for buf in out_buffers:
        res = unpack_output_vectorized(buf)
        results.append(res)
        
    # Clean up buffers
    for b in in_buffers: b.freebuffer()
    for b in out_buffers: b.freebuffer()
        
    return np.array(results), hw_time

In [ ]:
# Software FFT Processing Loop
def run_sw_fft(frames):
    start_time = time.perf_counter()
    # NumPy FFT is highly optimized
    results = np.fft.fft(frames, axis=1)
    end_time = time.perf_counter()
    return results, end_time - start_time

In [ ]:
# Run Benchmark
print("Running Hardware FFT...")
hw_results, hw_time = run_hw_fft(frames)
print(f"Hardware Time (DMA + Compute): {hw_time:.6f} s")
print(f"Hardware Throughput: {num_frames/hw_time:.2f} frames/s")

print("\nRunning Software FFT...")
sw_results, sw_time = run_sw_fft(frames)
print(f"Software Time: {sw_time:.6f} s")
print(f"Software Throughput: {num_frames/sw_time:.2f} frames/s")

print(f"\nSpeedup (SW/HW): {sw_time/hw_time:.2f}x")
print("Note: HW time now strictly measures DMA Transfer + FPGA Compute latency, excluding Python data packing overhead.")

In [ ]:
# Theoretical vs Measured Performance Analysis
# FPGA Clock Frequency: 100 MHz (defined in Vivado)
f_clk = 100e6 

# Theoretical Cycles per Frame
# Latency = (N / SSR) + Pipeline Depth
# N/SSR = 512 cycles. Pipeline depth is roughly 50-100 cycles for this IP.
cycles_per_frame = (N / SSR) + 80 
theoretical_time_per_frame = cycles_per_frame / f_clk

measured_time_per_frame = hw_time / num_frames

print("--- Performance Breakdown ---")
print(f"Theoretical FPGA Compute Time:   {theoretical_time_per_frame*1e6:.2f} us / frame")
print(f"Measured System Time (Python):   {measured_time_per_frame*1e6:.2f} us / frame")
print(f"Software Driver Overhead:        {(measured_time_per_frame - theoretical_time_per_frame)*1e6:.2f} us / frame")
print(f"Overhead Ratio:                  {((measured_time_per_frame/theoretical_time_per_frame)-1)*100:.1f}%")

print("\nConclusion:")
print("For small data sizes (N=1024), the Python driver overhead dominates the execution time.")
print("FPGA acceleration is most effective when:")
print("1. Processing much larger data frames.")
print("2. Using C/C++ drivers to minimize control overhead.")
print("3. Implementing the data source/sink directly in hardware (streaming).")

In [ ]:
# Advanced Analysis: Calibrating Loop Overhead
# ------------------------------------------------
# We can attempt to isolate the pure hardware + driver time by subtracting the 
# time spent in the Python loop mechanics (iteration and list access).

# 1. Measure the overhead of an identical loop without DMA calls
start_cal = time.perf_counter()
for i in range(num_frames):
    # Perform same indexing operations to mimic the loop structure
    # We use the 'frames' list which has the same length as the buffers list
    _ = frames[i] 
end_cal = time.perf_counter()

loop_overhead = end_cal - start_cal
corrected_hw_time = hw_time - loop_overhead

print(f"Original Measured Time:    {hw_time:.6f} s")
print(f"Python Loop Overhead:      {loop_overhead:.6f} s")
print(f"Corrected Hardware Time:   {corrected_hw_time:.6f} s")
print(f"Impact of Loop Overhead:   {(loop_overhead/hw_time)*100:.2f}%")

print("\nInsight:")
print("Even after removing the loop iteration cost, the time is likely still higher than the theoretical FPGA time.")
print("This is because the `dma.transfer()` function call itself involves significant CPU processing")
print("(setting up registers, cache flushing, etc.) which cannot be subtracted as it is required to drive the hardware.")

In [ ]:
# Verification and Visualization
frame_idx = num_frames // 2 # Pick a frame from the middle

plt.figure(figsize=(14, 8))

# Plot 1: Magnitude Comparison
plt.subplot(2, 1, 1)
plt.plot(np.abs(sw_results[frame_idx]), label="Software (NumPy)", alpha=0.7)
plt.plot(np.abs(hw_results[frame_idx]), '--', label="Hardware (FPGA)", alpha=0.7)
plt.title(f"FFT Magnitude Comparison (Frame {frame_idx})")
plt.legend()
plt.grid(True)

# Plot 2: Error
plt.subplot(2, 1, 2)
error = np.abs(np.abs(sw_results[frame_idx]) - np.abs(hw_results[frame_idx]))
plt.plot(error, color='red')
plt.title("Magnitude Error (SW - HW)")
plt.xlabel("Frequency Bin")
plt.ylabel("Difference")
plt.grid(True)

plt.tight_layout()
plt.show()